In [ ]:
"""\
# Copyright (C) 2024 Jesús Bautista Villar <jesbauti20@gmail.com>
"""
!python3 --version

In [ ]:
import numpy as np
import os
import sys
from tqdm import tqdm

# Graphic tools
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams["mathtext.fontset"] = "cm"
matplotlib.rc("font", **{"size" : 12})

# Animation tools
from IPython.display import HTML
from matplotlib.animation import PillowWriter

# -------------------------------------------------------------------------------------

# Swarm Systems Lab PySimUtils
from ssl_pysimutils import unicycle_patch, createDir, load_data

# GVF-IK python project to path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the GVF-IK simulator
from gvf_ik.gvf_traj.gvf_traj_ellipse import gvf_ellipse
from gvf_ik.simulator import simulator

# Import the animation classes
from gvf_ik.animations import AnimationXY, AnimationXYPhi, AnimationTelemetry

# -------------------------------------------------------------------------------------

# Data and output directories
DATA_PATH = os.path.join("..", "data")
OUTPUT_PATH = os.path.join("..", "output")

createDir(OUTPUT_PATH)

E = np.array([[0, 1],[-1, 0]])

## ICRA video animations

### Animation of the simulation

In [ ]:
# Initial state of the robot
p0 = np.array([[200,-200]])
v0 = np.array([14])
phi0 = np.array([-90])

x0 = [p0,v0,phi0]

# Generate the trajectory to be followed
a, b = 100, 70
XYoff, alpha = [0, 0], 0

gvf_traj = gvf_ellipse(XYoff,alpha,a,b)

# Generating the simulator class
dt = 0.1

s = 1
ke = 0.35
kn = 1

A, omega = 0, 0
sim = simulator(gvf_traj, x0=x0, dt=dt, s=s, ke=ke, kn=kn, A_fd=A, omega_fd=omega)

# ----------------------------------------------------------------------
# Execute the simulation given a desired final time stamp
# ----------------------------------------------------------------------
tf = 50

t_list = np.arange(0, tf, dt)
for it in tqdm(range(len(t_list))):
    sim.int_euler()

In [ ]:
animator = AnimationXY(gvf_traj, sim.data, fps=60, dpi=300, figsize=(7,6), 
                       xlims=[-200,200], ylims=[-200,200])

anim = animator.gen_animation()
HTML(anim.to_html5_video()) # It takes a loooot of time...

In [ ]:
animator = AnimationXYPhi(gvf_traj, sim.data, sim.ke, A, omega,
                          fps=60, dpi=300, figsize=(15,8), 
                          xlims=[-200,250], ylims=[-250,150], wait_period=3)

anim = animator.gen_animation()
HTML(anim.to_html5_video()) # It takes a loooot of time...

### Animation of the telemetry data

#### Experiment 1

In [ ]:
# Loading data
t0, tf, t0_cond, ke, kn = 1600+88, 1600+88+100, 8, 0.25, 0.6

filename = "24_09_11__10_04_38.csv"

data = load_data(os.path.join(DATA_PATH, filename), t0, tf)

# Hard code the followed trajectory
a, b = 100,100
XYoff, alpha = [-365, 254.2], 0
gvf_traj = gvf_ellipse(XYoff,alpha,a,b)

# Predicted data simulator parameters
sim_kw_args = {"gvf_traj": gvf_traj, "s":1, 
               "ke":ke, "kn":kn, "A_fd":0, "omega_fd":0}

In [ ]:
animator = AnimationTelemetry(data, sim_kw_args, dpi=300, figsize=(15,8), 
                              xlims=[-500,-100], ylims=[0,370], 
                              fps=60, wait_period=3, anim_tf=None)

anim = animator.gen_animation()
HTML(anim.to_html5_video()) # It takes a loooot of time...

#### Experiment 2

In [ ]:
# Loading data
t0, tf, t0_sin, omega = 1250, 1330, 6.55, 0.55  # kn=0.8, ke=0.25

filename = "24_09_11__10_48_44.csv"

data = load_data(os.path.join(DATA_PATH, filename), t0, tf)

# Hard code the followed trajectory
a, b = 100,100
XYoff, alpha = [-365, 254.2], 0
gvf_traj = gvf_ellipse(XYoff,alpha,a,b)

# Predicted data simulator parameters
sim_kw_args = {"gvf_traj": gvf_traj, "s":1, 
               "ke":0.25, "kn":0.8, "A_fd":0.1, "omega_fd":omega}

In [ ]:
animator = AnimationTelemetry(data, sim_kw_args, A=0.1, omega=omega, t0_sin=t0_sin,
                              dpi=100, figsize=(15,8), 
                              xlims=[XYoff[0]-150,XYoff[0]+150], 
                              ylims=[XYoff[1]-150,XYoff[1]+150],
                              ytick_sep_phi=0.1, ytick_sep_roll=0.2, 
                              fps=60, wait_period=0, anim_tf=None)

anim = animator.gen_animation()
HTML(anim.to_html5_video()) # It takes a loooot of time...